# Rosenbrock Function Optimization

## 1. CUDA Parallel Implementation

In [1]:
%%writefile rosenbrock_cuda.cu
#include <iostream>
#include <cmath>
#include <curand_kernel.h>
#include <cfloat>
#include <chrono>
#include <fstream>

using namespace std;

#define POP 256
#define DIM 5
#define MAX_IT 8000
#define LB -100.0
#define UB 100.0

#define BLOCK_SIZE 256

// Device function for generating random doubles
__device__ double randF(curandState* state, double a, double b) {
    double r = curand_uniform_double(state);
    return a + r * (b - a);
}

// Device function for generating random integers
__device__ int randInt(curandState* state, int a, int b) {
    float r = curand_uniform(state);
    return (int)(a + r * (b - a + 0.99999f));
}

// ------------------------------
// Benchmark Fitness Function (Rosenbrock)
// ------------------------------
__device__ double fitness(double* x, int dim) {
    double sum = 0.0;
    for (int i = 0; i < dim - 1; ++i) {
        double xi = x[i];
        double xi_next = x[i+1];
        sum += 100.0 * pow(xi_next - xi * xi, 2) + pow(1.0 - xi, 2);
    }
    return sum;
}

// ------------------------------
// CUDA Kernel: Initialize Population
// ------------------------------
__global__ void init_population_kernel(double* pop, curandState* states, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        curand_init(seed, idx, 0, &states[idx]);
        for (int d = 0; d < DIM; d++) {
            pop[idx * DIM + d] = randF(&states[idx], LB, UB);
        }
    }
}

// ------------------------------
// CUDA Kernel: Evaluate Fitness
// ------------------------------
__global__ void evaluate_fitness_kernel(double* pop, double* fitness_vals) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        double sol[DIM];
        for (int d = 0; d < DIM; d++) {
            sol[d] = pop[idx * DIM + d];
        }
        fitness_vals[idx] = fitness(sol, DIM);
    }
}

// ------------------------------
// CUDA Kernel: LOA Update Step
// ------------------------------
__global__ void loa_update_kernel(double* pop, double* fitness_vals,
                                  double* new_pop, double* new_fitness_vals,
                                  curandState* states, int t) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        curandState localState = states[idx];

        // 1. Find better candidates
        int betterIdx = -1;
        int betterCount = 0;

        for (int j = 0; j < POP; j++) {
            if (fitness_vals[j] < fitness_vals[idx]) {
                betterCount++;
            }
        }

        if (betterCount > 0) {
            int pick = randInt(&localState, 0, betterCount - 1);
            int current = 0;
            for (int j = 0; j < POP; j++) {
                if (fitness_vals[j] < fitness_vals[idx]) {
                    if (current == pick) {
                        betterIdx = j;
                        break;
                    }
                    current++;
                }
            }
        }

        // 2. Generate Candidate
        double candidate[DIM];
        double currentPos[DIM];
        for(int d=0; d<DIM; d++) currentPos[d] = pop[idx * DIM + d];

        double rp = curand_uniform_double(&localState);

        if (rp < 0.5 && betterIdx != -1) {
            // Escape Move
            double betterPos[DIM];
            for(int d=0; d<DIM; d++) betterPos[d] = pop[betterIdx * DIM + d];

            for (int d = 0; d < DIM; d++) {
                double r = curand_uniform_double(&localState);
                int I = randInt(&localState, 1, 2);
                candidate[d] = currentPos[d] + r * (betterPos[d] - I * currentPos[d]);

                // Bounds
                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        } else {
            // Hide Move
            for (int d = 0; d < DIM; d++) {
                double r = curand_uniform_double(&localState);
                candidate[d] = currentPos[d] + (1.0 - 2.0 * r) * (UB - LB) / (double)t;

                // Bounds
                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        }

        // 3. Evaluate Candidate
        double f = fitness(candidate, DIM);

        // 4. Update if better
        if (f < fitness_vals[idx]) {
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = candidate[d];
            new_fitness_vals[idx] = f;
        } else {
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = currentPos[d];
            new_fitness_vals[idx] = fitness_vals[idx];
        }

        states[idx] = localState; // Save state back
    }
}

int main() {
    auto t_start = chrono::high_resolution_clock::now();

    double *d_pop, *d_fitness, *d_new_pop, *d_new_fitness;
    curandState *d_states;

    size_t pop_size = POP * DIM * sizeof(double);
    size_t fit_size = POP * sizeof(double);

    cudaMalloc(&d_pop, pop_size);
    cudaMalloc(&d_fitness, fit_size);
    cudaMalloc(&d_new_pop, pop_size);
    cudaMalloc(&d_new_fitness, fit_size);
    cudaMalloc(&d_states, POP * sizeof(curandState));

    int threads = 128;
    int blocks = (POP + threads - 1) / threads;

    init_population_kernel<<<blocks, threads>>>(d_pop, d_states, time(NULL));
    evaluate_fitness_kernel<<<blocks, threads>>>(d_pop, d_fitness);
    cudaDeviceSynchronize();

    double bestGlobalFit = DBL_MAX;
    double h_fitness[POP];
    double h_pop[POP * DIM];
    double bestGlobalSol[DIM];

    // ---------- CSV LOGGER ----------
    ofstream csv("rosenbrock_cuda_log.csv");
    csv << "iter,best";
    for(int d=1; d<=DIM; d++) csv << ",x" << d;
    csv << "\n";

    // Main Loop
    for (int it = 1; it <= MAX_IT; it++) {
        loa_update_kernel<<<blocks, threads>>>(d_pop, d_fitness,
                                               d_new_pop, d_new_fitness,
                                               d_states, it);
        cudaDeviceSynchronize();

        // Swap pointers
        double *temp_pop = d_pop; d_pop = d_new_pop; d_new_pop = temp_pop;
        double *temp_fit = d_fitness; d_fitness = d_new_fitness; d_new_fitness = temp_fit;

        // Copy to host EVERY iteration for CSV + best tracking
        cudaMemcpy(h_fitness, d_fitness, fit_size, cudaMemcpyDeviceToHost);
        cudaMemcpy(h_pop,     d_pop,     pop_size, cudaMemcpyDeviceToHost);

        // Find best in current population
        int bestIdx = 0;
        double bestIterFit = h_fitness[0];
        for(int i=1; i<POP; i++) {
            if(h_fitness[i] < bestIterFit) {
                bestIterFit = h_fitness[i];
                bestIdx = i;
            }
        }

        // Update global best
        if(bestIterFit < bestGlobalFit) {
            bestGlobalFit = bestIterFit;
            for(int d=0; d<DIM; d++)
                bestGlobalSol[d] = h_pop[bestIdx * DIM + d];
        }

        // CSV: write GLOBAL best so far
        csv << it << "," << bestGlobalFit;
        for(int d=0; d<DIM; d++)
            csv << "," << bestGlobalSol[d];
        csv << "\n";

        // Printing same as before (every 1000 iters)
        if (it % 1000 == 0) {
             printf("Iter %d | Best = %f\n", it, bestGlobalFit);
        }
    }

    csv.close();

    // Final Result (from tracked global best)
    printf("\nFinal Best Solution:\n");
    for(int d=0; d<DIM; d++) {
        printf("x%d = %f\n", d+1, bestGlobalSol[d]);
    }
    printf("\nBest Rosenbrock Value = %f\n", bestGlobalFit);

    cudaFree(d_pop);
    cudaFree(d_fitness);
    cudaFree(d_new_pop);
    cudaFree(d_new_fitness);
    cudaFree(d_states);

    auto t_end = chrono::high_resolution_clock::now();
    double elapsed = chrono::duration_cast<chrono::duration<double>>(t_end - t_start).count();
    printf("\nExecution Time = %f sec\n", elapsed);
    printf("CSV log saved as rosenbrock_cuda_log.csv\n");

    return 0;
}


Writing rosenbrock_cuda.cu


In [3]:
!nvcc -arch=sm_75 rosenbrock_cuda.cu -o rosenbrock_cuda
!./rosenbrock_cuda

Iter 1000 | Best = 0.642682
Iter 2000 | Best = 0.349110
Iter 3000 | Best = 0.244967
Iter 4000 | Best = 0.211279
Iter 5000 | Best = 0.161227
Iter 6000 | Best = 0.123194
Iter 7000 | Best = 0.098993
Iter 8000 | Best = 0.065349

Final Best Solution:
x1 = 0.972682
x2 = 0.943343
x3 = 0.886193
x4 = 0.785282
x5 = 0.615258

Best Rosenbrock Value = 0.065349

Execution Time = 1.201019 sec
CSV log saved as rosenbrock_cuda_log.csv


## 2. Serial (Sequential) Implementation

In [4]:

%%writefile rosenbrock_serial.cpp
#include <bits/stdc++.h>
using namespace std;

/* ---------------------------------------
      Random Generator
---------------------------------------*/
mt19937 rng(time(NULL));

double randF(double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double rosenbrock(const vector<double> &x) {
    int dim = x.size();

    double sum = 0.0;
    for (int i = 0; i < dim - 1; ++i) {
        double xi = x[i];
        double xi_next = x[i+1];
        sum += 100.0 * pow(xi_next - xi * xi, 2) + pow(1.0 - xi, 2);
    }
    return sum;

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(LB, UB); // Note: Original code used randF(rng,0,1) here but logic was r*(SSA-I*x).
        // Wait, original code: double r = randF(rng,0,1);
        // Let's stick to original logic.
        double r_val = randF(0, 1);
        int I = randInt(1, 2);
        newX[j] = x[j] + r_val * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search)
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(0, 1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population
---------------------------------------*/
vector<vector<double>> init_population(){
    vector<vector<double>> pop(POP, vector<double>(DIM));
    for(int i=0;i<POP;i++)
        for(int d=0;d<DIM;d++)
            pop[i][d] = randF(LB, UB);
    return pop;
}

/* =======================================
             MAIN LOA SERIAL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    vector<vector<double>> pop = init_population();
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=rosenbrock(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        for(int i=0;i<POP;i++){

            vector<int> better;
            for(int j=0;j<POP;j++)
                if(fitness[j]<fitness[i]) better.push_back(j);

            int betterIdx=-1;
            if(!better.empty())
                betterIdx = better[randInt(0,(int)better.size()-1)];

            vector<double> candidate;
            if(randF(0,1)<0.5 && betterIdx!=-1)
                candidate = escape(pop[i],pop[betterIdx]);
            else
                candidate = hide(pop[i],it);

            double f = rosenbrock(candidate);

            if(f<fitness[i]){
                pop[i]=candidate;
                fitness[i]=f;
            }
            if(f<bestFit){
                bestFit=f;
                bestSol=candidate;
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Rosenbrock" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Writing rosenbrock_serial.cpp


In [5]:
!g++ rosenbrock_serial.cpp -o rosenbrock_serial
!./rosenbrock_serial

Iter 1000 | Best = 0.908692
Iter 2000 | Best = 0.315965
Iter 3000 | Best = 0.192591
Iter 4000 | Best = 0.15899
Iter 5000 | Best = 0.146628
Iter 6000 | Best = 0.119559
Iter 7000 | Best = 0.0763107
Iter 8000 | Best = 0.0703504

Final Best Solution:
x1 = 0.969371
x2 = 0.93872
x3 = 0.881304
x4 = 0.774753
x5 = 0.598342

Best Rosenbrock Value = 0.0703504

Execution Time = 17.2656 sec


## 3. OpenMP Parallel Implementation

In [6]:

%%writefile rosenbrock_omp.cpp
#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

/* ---------------------------------------
      Thread-safe Random Generator
---------------------------------------*/
double randF(mt19937 &rng, double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(mt19937 &rng, int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double rosenbrock(const vector<double> &x) {
    int dim = x.size();

    double sum = 0.0;
    for (int i = 0; i < dim - 1; ++i) {
        double xi = x[i];
        double xi_next = x[i+1];
        sum += 100.0 * pow(xi_next - xi * xi, 2) + pow(1.0 - xi, 2);
    }
    return sum;

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)  — parallel safe
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA,
                      mt19937 &rng)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        int I = randInt(rng,1,2);
        newX[j] = x[j] + r * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search) — parallel safe
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t,mt19937 &rng){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population (PARALLEL)
---------------------------------------*/
vector<vector<double>> init_population(vector<mt19937> &rngs){
    vector<vector<double>> pop(POP, vector<double>(DIM));

    #pragma omp parallel
    {
        int tid = omp_get_thread_num();
        mt19937 &local_rng = rngs[tid];

        #pragma omp for schedule(static)
        for(int i=0;i<POP;i++)
            for(int d=0;d<DIM;d++)
                pop[i][d] = randF(local_rng,LB,UB);
    }
    return pop;
}

/* =======================================
             MAIN LOA PARALLEL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    int threads = omp_get_max_threads();
    vector<mt19937> rngs(threads);

    random_device rd;
    for(int i=0;i<threads;i++)
        rngs[i].seed(rd()+i*111);

    vector<vector<double>> pop = init_population(rngs);
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=rosenbrock(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
       Full population parallel every step
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        #pragma omp parallel
        {
            int tid = omp_get_thread_num();
            mt19937 &localRng = rngs[tid];

            double localBest = 1e18;
            vector<double> localBestSol(DIM);

            #pragma omp for schedule(static)
            for(int i=0;i<POP;i++){

                vector<int> better;
                for(int j=0;j<POP;j++)
                    if(fitness[j]<fitness[i]) better.push_back(j);

                int betterIdx=-1;
                if(!better.empty())
                    betterIdx = better[randInt(localRng,0,(int)better.size()-1)];

                vector<double> candidate;
                if(randF(localRng,0,1)<0.5 && betterIdx!=-1)
                    candidate = escape(pop[i],pop[betterIdx],localRng);
                else
                    candidate = hide(pop[i],it,localRng);

                double f = rosenbrock(candidate);

                if(f<fitness[i]){
                    pop[i]=candidate;
                    fitness[i]=f;
                }
                if(f<localBest){
                    localBest=f;
                    localBestSol=candidate;
                }
            }

            // Update global best safely
            #pragma omp critical
            {
                if(localBest<bestFit){
                    bestFit=localBest;
                    bestSol=localBestSol;
                }
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Rosenbrock" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Writing rosenbrock_omp.cpp


In [7]:
!g++ -fopenmp rosenbrock_omp.cpp -o rosenbrock_omp
!./rosenbrock_omp

Iter 1000 | Best = 0.843594
Iter 2000 | Best = 0.385791
Iter 3000 | Best = 0.241422
Iter 4000 | Best = 0.164788
Iter 5000 | Best = 0.109868
Iter 6000 | Best = 0.0997058
Iter 7000 | Best = 0.0860853
Iter 8000 | Best = 0.0707731

Final Best Solution:
x1 = 0.968278
x2 = 0.937733
x3 = 0.88012
x4 = 0.774266
x5 = 0.59728

Best Rosenbrock Value = 0.0707731

Execution Time = 13.4611 sec
